In [54]:
import os
import torch

from trainer import Trainer, TrainerArgs
from TTS.config.shared_configs import BaseAudioConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.tacotron2_config import Tacotron2Config
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.tacotron2 import Tacotron2
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

In [56]:
output_path = "/Users/neil/Code/TTS_GI/outputs"

# init configs

In [59]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech",
    meta_file_train="/Users/neil/Code/TTS_GI/dataset/metadata.csv",
    path="/Users/neil/Code/TTS_GI/dataset",
)

audio_config = BaseAudioConfig(
    sample_rate=16000,
    do_trim_silence=True,
    trim_db=60.0,
    signal_norm=False,
    mel_fmin=0.0,
    mel_fmax=8000,
    spec_gain=1.0,
    log_func="np.log",
    ref_level_db=20,
    preemphasis=0.0,
)    

config = Tacotron2Config(
    audio=audio_config,
    batch_size=64,
    eval_batch_size=16, 
    num_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    r=6,
    gradual_training=[[0, 6, 64], [10000, 4, 32], [50000, 3, 32], [100000, 2, 32]],
    double_decoder_consistency=True,
    epochs=1000,
    text_cleaner="chinese_mandarin_cleaners",
    use_phonemes=True,
    phoneme_language="zh-cn",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    precompute_num_workers=8,
    print_step=25,
    print_eval=True,
    mixed_precision=False,
    output_path=output_path,
    datasets=[dataset_config],
)

# init audio processor

In [60]:
ap = AudioProcessor(**config.audio)
ap = AudioProcessor.init_from_config(config)

tokenizer, config = TTSTokenizer.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60.0
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5


# Load Dataset

In [61]:
train_samples, val_samples = load_tts_samples(
    dataset_config, 
    eval_split=0.3,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

print(f"Train samples: {len(train_samples)}")
print(f"Val samples: {len(val_samples)}")
print(f"Train Sample: {train_samples[5]}")
print(f"Val Sample: {val_samples[5]}")

 | > Found 697 files in /Users/neil/Code/TTS_GI/dataset
Train samples: 691
Val samples: 6
Train Sample: {'text': '木南小姐，你好。\n', 'audio_file': '/Users/neil/Code/TTS_GI/dataset/wavs/dataset/wavs/神里绫华_train_390.wav.wav', 'speaker_name': 'ljspeech', 'root_path': '/Users/neil/Code/TTS_GI/dataset', 'language': '', 'audio_unique_name': '#wavs/dataset/wavs/神里绫华_train_390.wav'}
Val Sample: {'text': '风花雪月，虽然作为我们熟知的事物存在于世界之中，但也应有属于它们自己的「感情」。\n', 'audio_file': '/Users/neil/Code/TTS_GI/dataset/wavs/dataset/wavs/神里绫华_train_31.wav.wav', 'speaker_name': 'ljspeech', 'root_path': '/Users/neil/Code/TTS_GI/dataset', 'language': '', 'audio_unique_name': '#wavs/dataset/wavs/神里绫华_train_31.wav'}


In [45]:
pretrained_model_path = "/Users/neil/Code/TTS_GI/outputs/pretrain_models/model_file.pth"
model = Tacotron2(config, ap, tokenizer)
model.load_state_dict(torch.load(pretrained_model_path, map_location=torch.device("cpu")), strict=False)

trainer = Trainer(
    TrainerArgs(), 
    config, 
    output_path, 
    model=model,
    train_samples=train_samples,
    eval_samples=val_samples
)

trainer.fit()

StopIteration: 

In [63]:
model = Tacotron2(config, ap, tokenizer, speaker_manager=None)

# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=val_samples
)
trainer.fit()

StopIteration: 